In [102]:
import pandas as pd
import numpy as np
import xgboost as xgb
from math import log
from sklearn.metrics import log_loss, cohen_kappa_score, confusion_matrix

In [104]:
from hyperband import Hyperband
from defs.gb import get_params, try_params

ModuleNotFoundError: No module named 'hyperband'

In [72]:
data = pd.read_csv('../data/HR_comma_sep.csv')

In [73]:
print(data.shape)
data[:5]

(14999, 10)


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [82]:
data = pd.get_dummies(data)

In [83]:
sum(data['left'])/data.shape[0]

0.2380825388359224

In [84]:
feats = [col for col in data.columns if col != 'left']

In [85]:
X_train, Y_train = data.loc[int(len(X)*0.8):, feats], data.loc[int(len(X)*0.8):, 'left']
X_test, Y_test = data.loc[:int(len(X)*0.8), feats], data.loc[:int(len(X)*0.8), 'left']

In [86]:
R = 81
eta = 3
s_max = log(R, eta)
B = (s_max +1)*R
s = np.arange(s_max)
n = B/R * (eta**s[0]/(s+1))
r = int(R*(eta**(-s[0])))
print(s_max)
print(B)
print(r)

4.0
405.0
81


In [97]:
bst = xgb.XGBClassifier(n_estimators=r)
bst.fit(X_train, Y_train)
probs = bst.predict_proba(X_test)[:,1]
preds = 1*(probs > 0.5)
loss = cohen_kappa_score(preds, Y_test)
#confusion_matrix(preds, Y_test)

array([[  0,  63],
       [  0, 738]])

In [ ]:
def get_params():
    space = {
        'learning_rate': np.random.uniform(0.005, 0.3, 1),
        'max_depth': int(np.random.uniform(2, 15, 1)),
        'min_child_weight': int(np.random.uniform(1, 10, 1)),
        'subsample': 
        hp.choice( 'ss', [
		'default',
		hp.uniform( 'ss_', 0.5, 1.0 )
	]),
	'colsample_bytree': hp.choice( 'cbt', [
		'default',
		hp.uniform( 'cbt_', 0.5, 1.0 )
	]),
	'colsample_bylevel': hp.choice( 'cbl', [
		'default',
		hp.uniform( 'cbl_', 0.5, 1.0 )
	]),	
	'gamma': hp.choice( 'g', [
		'default',
		hp.uniform( 'g_', 0, 1 )
	]),
	'reg_alpha': hp.choice( 'ra', [
		'default',
		hp.loguniform( 'ra_', log( 1e-10 ), log( 1 ))
	]),
	'reg_lambda': hp.choice( 'rl', [
		'default',
		hp.uniform( 'rl_', 0.1, 10 )
	]),
	'base_score': hp.choice( 'bs', [
		'default',
		hp.uniform( 'bs_', 0.1, 0.9 )
	]),
	'scale_pos_weight': hp.choice( 'spw', [
		'default',
		hp.uniform( 'spw', 0.1, 10 )
	])
}